
<a id='text'></a>
{{ header }}

# Working with text data


<a id='text-string-methods'></a>
Series and Index are equipped with a set of string processing methods
that make it easy to operate on each element of the array. Perhaps most
importantly, these methods exclude missing/NA values automatically. These are
accessed via the `str` attribute and generally have names matching
the equivalent (scalar) built-in string methods:

The string methods on Index are especially useful for cleaning up or
transforming DataFrame columns. For instance, you may have columns with
leading or trailing whitespace:

Since `df.columns` is an Index object, we can use the `.str` accessor

These string methods can then be used to clean up the columns as needed.
Here we are removing leading and trailing whitespaces, lower casing all names,
and replacing any remaining whitespaces with underscores:

>**Note**
>
>If you have a `Series` where lots of elements are repeated
(i.e. the number of unique elements in the `Series` is a lot smaller than the length of the
`Series`), it can be faster to convert the original `Series` to one of type
`category` and then use `.str.<method>` or `.dt.<property>` on that.
The performance difference comes from the fact that, for `Series` of type `category`, the
string operations are done on the `.categories` and not on each element of the
`Series`.

Please note that a `Series` of type `category` with string `.categories` has
some limitations in comparison to `Series` of type string (e.g. you can’t add strings to
each other: `s + " " + s` won’t work if `s` is a `Series` of type `category`). Also,
`.str` methods which operate on elements of type `list` are not available on such a
`Series`.


<a id='text-warn-types'></a>
Before v.0.25.0, the `.str`-accessor did only the most rudimentary type checks. Starting with
v.0.25.0, the type of the Series is inferred and the allowed types (i.e. strings) are enforced more rigorously.

Generally speaking, the `.str` accessor is intended to work only on strings. With very few
exceptions, other uses are not supported, and may be disabled at a later point.

## Splitting and replacing strings


<a id='text-split'></a>
Methods like `split` return a Series of lists:

Elements in the split lists can be accessed using `get` or `[]` notation:

It is easy to expand this to return a DataFrame using `expand`.

It is also possible to limit the number of splits:

`rsplit` is similar to `split` except it works in the reverse direction,
i.e., from the end of the string to the beginning of the string:

`replace` by default replaces [regular expressions](https://docs.python.org/3/library/re.html):

Some caution must be taken to keep regular expressions in mind! For example, the
following code will cause trouble because of the regular expression meaning of
\$:

New in version 0.23.0.

If you do want literal replacement of a string (equivalent to
[`str.replace()`](https://docs.python.org/3/library/stdtypes.html#str.replace)), you can set the optional `regex` parameter to
`False`, rather than escaping each character. In this case both `pat`
and `repl` must be strings:

New in version 0.20.0.

The `replace` method can also take a callable as replacement. It is called
on every `pat` using [`re.sub()`](https://docs.python.org/3/library/re.html#re.sub). The callable should expect one
positional argument (a regex object) and return a string.

New in version 0.20.0.

The `replace` method also accepts a compiled regular expression object
from [`re.compile()`](https://docs.python.org/3/library/re.html#re.compile) as a pattern. All flags should be included in the
compiled regular expression object.

Including a `flags` argument when calling `replace` with a compiled
regular expression object will raise a `ValueError`.


<a id='text-concatenate'></a>

## Concatenation

There are several ways to concatenate a `Series` or `Index`, either with itself or others, all based on `cat()`,
resp. `Index.str.cat`.

### Concatenating a single Series into a string

The content of a `Series` (or `Index`) can be concatenated:

If not specified, the keyword `sep` for the separator defaults to the empty string, `sep=''`:

By default, missing values are ignored. Using `na_rep`, they can be given a representation:

### Concatenating a Series and something list-like into a Series

The first argument to `cat()` can be a list-like object, provided that it matches the length of the calling `Series` (or `Index`).

Missing values on either side will result in missing values in the result as well, *unless* `na_rep` is specified:

### Concatenating a Series and something array-like into a Series

New in version 0.23.0.

The parameter `others` can also be two-dimensional. In this case, the number or rows must match the lengths of the calling `Series` (or `Index`).

### Concatenating a Series and an indexed object into a Series, with alignment

New in version 0.23.0.

For concatenation with a `Series` or `DataFrame`, it is possible to align the indexes before concatenation by setting
the `join`-keyword.

If the `join` keyword is not passed, the method `cat()` will currently fall back to the behavior before version 0.23.0 (i.e. no alignment),
but a `FutureWarning` will be raised if any of the involved indexes differ, since this default will change to `join='left'` in a future version.

The usual options are available for `join` (one of `'left', 'outer', 'inner', 'right'`).
In particular, alignment also means that the different lengths do not need to coincide anymore.

The same alignment can be used when `others` is a `DataFrame`:

### Concatenating a Series and many objects into a Series

Several array-like items (specifically: `Series`, `Index`, and 1-dimensional variants of `np.ndarray`)
can be combined in a list-like container (including iterators, `dict`-views, etc.).

All elements without an index (e.g. `np.ndarray`) within the passed list-like must match in length to the calling `Series` (or `Index`),
but `Series` and `Index` may have arbitrary length (as long as alignment is not disabled with `join=None`):

If using `join='right'` on a list-like of `others` that contains different indexes,
the union of these indexes will be used as the basis for the final concatenation:

## Indexing with `.str`


<a id='text-indexing'></a>
You can use `[]` notation to directly index by position locations. If you index past the end
of the string, the result will be a `NaN`.

## Extracting substrings


<a id='text-extract'></a>

### Extract first match in each subject (extract)

In version 0.18.0, `extract` gained the `expand` argument. When
`expand=False` it returns a `Series`, `Index`, or
`DataFrame`, depending on the subject and regular expression
pattern (same behavior as pre-0.18.0). When `expand=True` it
always returns a `DataFrame`, which is more consistent and less
confusing from the perspective of a user. `expand=True` is the
default since version 0.23.0.

The `extract` method accepts a [regular expression](https://docs.python.org/3/library/re.html) with at least one
capture group.

Extracting a regular expression with more than one group returns a
DataFrame with one column per group.

Elements that do not match return a row filled with `NaN`. Thus, a
Series of messy strings can be “converted” into a like-indexed Series
or DataFrame of cleaned-up or more useful strings, without
necessitating `get()` to access tuples or `re.match` objects. The
dtype of the result is always object, even if no match is found and
the result only contains `NaN`.

Named groups like

and optional groups like

can also be used. Note that any capture group names in the regular
expression will be used for column names; otherwise capture group
numbers will be used.

Extracting a regular expression with one group returns a `DataFrame`
with one column if `expand=True`.

It returns a Series if `expand=False`.

Calling on an `Index` with a regex with exactly one capture group
returns a `DataFrame` with one column if `expand=True`.

It returns an `Index` if `expand=False`.

Calling on an `Index` with a regex with more than one capture group
returns a `DataFrame` if `expand=True`.

It raises `ValueError` if `expand=False`.

In [ ]:
>>> s.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=False)
ValueError: only one regex group is supported with Index

The table below summarizes the behavior of `extract(expand=False)`
(input subject in first column, number of groups in regex in
first row)

||1 group|>1 group|
|Index|Index|ValueError|
|Series|Series|DataFrame|

### Extract all matches in each subject (extractall)


<a id='text-extractall'></a>
New in version 0.18.0.

Unlike `extract` (which returns only the first match),

the `extractall` method returns every match. The result of
`extractall` is always a `DataFrame` with a `MultiIndex` on its
rows. The last level of the `MultiIndex` is named `match` and
indicates the order in the subject.

When each subject string in the Series has exactly one match,

then `extractall(pat).xs(0, level='match')` gives the same result as
`extract(pat)`.

`Index` also supports `.str.extractall`. It returns a `DataFrame` which has the
same result as a `Series.str.extractall` with a default index (starts from 0).

New in version 0.19.0.

## Testing for Strings that match or contain a pattern

You can check whether elements contain a pattern:

Or whether elements match a pattern:

The distinction between `match` and `contains` is strictness: `match`
relies on strict `re.match`, while `contains` relies on `re.search`.

Methods like `match`, `contains`, `startswith`, and `endswith` take
an extra `na` argument so missing values can be considered True or False:


<a id='text-indicator'></a>

## Creating indicator variables

You can extract dummy variables from string columns.
For example if they are separated by a `'|'`:

String `Index` also supports `get_dummies` which returns a `MultiIndex`.

New in version 0.18.1.

See also `get_dummies()`.

## Method summary


<a id='text-summary'></a>
````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````|Method|Description|
|:------------------:|:------------------------------------------------------------------------------:|
|cat()|Concatenate strings|
|split()|Split strings on delimiter|
|rsplit()|Split strings on delimiter working from the end of the string|
|get()|Index into each element (retrieve i-th element)|
|join()|Join strings in each element of the Series with passed separator|
|get_dummies()|Split strings on the delimiter returning DataFrame of dummy variables|
|contains()|Return boolean array if each string contains pattern/regex|
|replace()|Replace occurrences of pattern/regex/string with some other string or the return value of a callable given the occurrence|
|repeat()|Duplicate values (s.str.repeat(3) equivalent to x * 3)|
|pad()|Add whitespace to left, right, or both sides of strings|
|center()|Equivalent to str.center|
|ljust()|Equivalent to str.ljust|
|rjust()|Equivalent to str.rjust|
|zfill()|Equivalent to str.zfill|
|wrap()|Split long strings into lines with length less than a given width|
|slice()|Slice each string in the Series|
|slice_replace()|Replace slice in each string with passed value|
|count()|Count occurrences of pattern|
|startswith()|Equivalent to str.startswith(pat) for each element|
|endswith()|Equivalent to str.endswith(pat) for each element|
|findall()|Compute list of all occurrences of pattern/regex for each string|
|match()|Call re.match on each element, returning matched groups as list|
|extract()|Call re.search on each element, returning DataFrame with one row for each element and one column for each regex capture group|
|extractall()|Call re.findall on each element, returning DataFrame with one row for each match and one column for each regex capture group|
|len()|Compute string lengths|
|strip()|Equivalent to str.strip|
|rstrip()|Equivalent to str.rstrip|
|lstrip()|Equivalent to str.lstrip|
|partition()|Equivalent to str.partition|
|rpartition()|Equivalent to str.rpartition|
|lower()|Equivalent to str.lower|
|casefold()|Equivalent to str.casefold|
|upper()|Equivalent to str.upper|
|find()|Equivalent to str.find|
|rfind()|Equivalent to str.rfind|
|index()|Equivalent to str.index|
|rindex()|Equivalent to str.rindex|
|capitalize()|Equivalent to str.capitalize|
|swapcase()|Equivalent to str.swapcase|
|normalize()|Return Unicode normal form. Equivalent to unicodedata.normalize|
|translate()|Equivalent to str.translate|
|isalnum()|Equivalent to str.isalnum|
|isalpha()|Equivalent to str.isalpha|
|isdigit()|Equivalent to str.isdigit|
|isspace()|Equivalent to str.isspace|
|islower()|Equivalent to str.islower|
|isupper()|Equivalent to str.isupper|
|istitle()|Equivalent to str.istitle|
|isnumeric()|Equivalent to str.isnumeric|
|isdecimal()|Equivalent to str.isdecimal|